In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils.vasp.database import create_summary
from pymatgen.core import Structure

from utils.potential_tools.generic import get_potential_df
from utils.potential_tools.ace import get_ace_fitting_df
from utils.potential_tools.magnetism import classify_magnetism_df

from utils.potential_tools.plotters import plot_magnetic_classification_distribution, plot_potential_data_summary
from glob import glob
import os

from pyiron_workflow import Workflow

%load_ext autoreload
%autoreload 2

In [2]:
import pyiron_workflow as pwf

In [3]:
from ase.optimize import BFGS
from ase import Atoms
from ase.calculators.calculator import Calculator
from pymatgen.io.ase import AseAtomsAdaptor

def calc_structure(structure,
                   calc: Calculator,
                   fmax: float = 0.01,
                   max_steps: int = 10000,
                   properties=('energy', 'forces', 'stresses')):
    """
    Relax an ASE atoms object with an ASE Calculator, and return only the requested properties.

    Args:
        structure: Pymatgen Structure
        calc: an ASE Calculator instance
        fmax: force convergence criterion (eV/Å)
        max_steps: maximum ionic steps
        properties: tuple/list of property names to return; choices include:
            'energy', 'forces', 'stresses',
            'positions', 'numbers', 'masses',
            'cell', 'volume',
            'charges', 'dipole', 'magmoms',
            'virial', 'pressure'
    Returns:
        atoms: relaxed ASE Atoms object
        results: dict mapping each requested name → array or scalar
        converged: True if optimizer converged, else False
    """

    # Convert to ASE Atoms and attach calculator
    atoms = structure.copy()#AseAtomsAdaptor.get_atoms(structure)
    atoms.calc = calc

    # Run relaxation
    optimizer = BFGS(atoms)
    converged = optimizer.run(fmax=fmax, steps=max_steps)

    # Gather all possible results, skipping any non‐implemented ones
    all_results = {}

    # Always-available
    all_results['energy']   = atoms.get_potential_energy()
    all_results['forces']   = atoms.get_forces()

    # Wrap stresses in try/except too
    try:
        all_results['stresses'] = atoms.get_stress()
    except (NotImplementedError, AttributeError):
        pass

    all_results['cell']      = atoms.get_cell()
    all_results['volume']    = atoms.get_volume()
    all_results['positions'] = atoms.get_positions()
    all_results['numbers']   = atoms.get_atomic_numbers()
    all_results['masses']    = atoms.get_masses()

    # Optional: charges, dipole, magmoms
    try:
        all_results['charges'] = atoms.get_charges()
    except (NotImplementedError, AttributeError):
        pass

    try:
        all_results['dipole'] = atoms.get_dipole_moment()
    except (NotImplementedError, AttributeError):
        pass

    try:
        all_results['magmoms'] = atoms.get_magnetic_moments()
    except (NotImplementedError, AttributeError):
        pass

    # Optional: virial and pressure
    try:
        all_results['virial']   = atoms.get_virial()
        all_results['pressure'] = atoms.get_pressure()
    except (NotImplementedError, AttributeError):
        pass

    # Filter only the properties the user asked for
    results = {}
    for prop in properties:
        if prop not in all_results:
            raise ValueError(
                f"Unknown or unsupported property '{prop}'. "
                f"Available = {list(all_results)}"
            )
        results[prop] = all_results[prop]

    return atoms, results, converged


In [4]:
import numpy as np
from ase import Atoms
import pyiron_workflow as pwf

@pwf.as_function_node("structure_list")
def generate_structures(
    base_structure: Atoms,
    axes: list[str] = ["iso"],
    strain_range: tuple[float, float] = (-0.2, 0.2),
    num_points: int = 11,
) -> list[Atoms]:
    """
    Generate a list of strained ASE Atoms structures.

    Parameters
    ----------
    base_structure
        ASE Atoms object to be strained.
    axes
        List of axes to strain simultaneously: any combination of "x", "y", "z".
        Use ["iso"] for isotropic (all axes) by default.
    strain_range
        (min_strain, max_strain), e.g. (-0.2, 0.2) for ±20%.
    num_points
        Number of steps in the strain grid.

    Returns
    -------
    List of ASE Atoms, one per epsilon value with specified axes strained.
    """
    structure_list: list[Atoms] = []
    start, end = strain_range

    for epsilon in np.linspace(start, end, num_points):
        s = base_structure.copy()
        cell = s.get_cell()

        # isotropic if requested
        if "iso" in [ax.lower() for ax in axes]:
            new_cell = cell * (1 + epsilon)
        else:
            new_cell = cell.copy()
            for ax in axes:
                ax_lower = ax.lower()
                if ax_lower == "x":
                    new_cell[0] = cell[0] * (1 + epsilon)
                elif ax_lower == "y":
                    new_cell[1] = cell[1] * (1 + epsilon)
                elif ax_lower == "z":
                    new_cell[2] = cell[2] * (1 + epsilon)
                else:
                    # ignore unknown axis labels
                    continue

        s.set_cell(new_cell, scale_atoms=True)
        structure_list.append(s)

    return structure_list


@pwf.as_function_node("e0", "v0", "B")
def equation_of_state(energies, volumes, eos="sj"):
    from ase.eos import EquationOfState
    eos = EquationOfState(volumes, energies, eos=eos)
    v0, e0, B = eos.fit() #v0, e0, B
    return e0, v0, B#eos_results

# Evaluates structures in a single node in serial
@pwf.as_function_node("structures", "results_dict", "convergence_lst")
def evaluate_structures(structures, calc, calc_kwargs):
    rel_structs_lst = []
    results_lst = []
    convergence_lst = []
    for struct in structures:
        rel_struct, result, convergence = calc_structure(struct, calc, **calc_kwargs)
        rel_structs_lst.append(rel_struct)
        results_lst.append(result)
        convergence_lst.append(convergence)
    return rel_structs_lst, results_lst, convergence_lst

@pwf.as_function_node("energies", "volumes")
def extract_energies_volumes_from_output(results, energy_parser_func, energy_parser_func_kwargs, volume_parser_func, volume_parser_func_kwargs):
    energies = energy_parser_func(results, **energy_parser_func_kwargs)
    volumes = volume_parser_func(results, **volume_parser_func_kwargs)
    return energies, volumes



In [5]:
@pwf.as_function_node("output")
def extract_values(results_list, key):
    """
    Extract a list of values for a specified key from a list of result dictionaries.

    Parameters
    ----------
    results_list : list of dict
        Each dict should contain the specified key.
    key : str
        The dictionary key to extract values for (e.g., 'energy', 'volume').

    Returns
    -------
    values : list
        List of values corresponding to `key` from each dict.

    Raises
    ------
    KeyError
        If any entry in results_list is missing the specified key.
    """
    try:
        extracted_values = [entry[key] for entry in results_list]
    except Exception as e:
        print(f"Error {e} when trying to parse output")
        extracted_values = np.nan
    return extracted_values

In [6]:
from tensorpotential.calculator import TPCalculator
#df = pd.read_pickle("/cmmc/u/hmai/FePotentials/generate_extxyz_2025_01_06/2025_01_17_FeGBPtableProject_PotentialComparison.pkl.gz")
calc = TPCalculator("/cmmc/u/hmai/FePotentials/potential_files/GRACE/2025_04_28_Fe_FeC_C_pyxtal/final_model/")

wf = Workflow("EOS")
from ase.build import bulk
bulk_Fe = bulk("Fe", cubic=True, a=2.83)
bulk_Fe.rattle()
for n in wf:
    n.use_cache = False

wf.structures_list = generate_structures(bulk_Fe,
                                    axes=["x", "y", "z"],
                                    strain_range=(-0.2, 0.2),
                                    num_points=11)
wf.evaluation = evaluate_structures(structures = wf.structures_list,
                                    calc=calc, 
                                    calc_kwargs = {"fmax": 0.01, "max_steps": 10000, "properties":('energy', 'forces', 'stresses', 'volume')})
wf.energies = extract_values(wf.evaluation.outputs.results_dict, key="energy")
wf.volumes = extract_values(wf.evaluation.outputs.results_dict, key="volume")

wf.eos = equation_of_state(wf.energies, wf.volumes, eos="birchmurnaghan")

# wf.energy_volume_data = extract_energies_volumes_from_output(results = wf.evaluation.outputs.results_dict,
#                                                              energy_parser_func = extract_values,
#                                                              energy_parser_func_kwargs = {"key": "energy"},
#                                                              volume_parser_func = extract_values,
#                                                              volume_parser_func_kwargs = {"key": "volume"})
wf.run()

2025-04-30 15:54:00.916527: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-30 15:54:00.920290: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-30 15:54:00.925135: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-30 15:54:00.940450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-30 15:54:00.968165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

      Step     Time          Energy          fmax
BFGS:    0 15:54:07       -5.863933        0.019501
BFGS:    1 15:54:07       -5.863941        0.009489
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -10.897777        0.018871
BFGS:    1 15:54:07      -10.897785        0.009942
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -13.423383        0.020728
BFGS:    1 15:54:07      -13.423393        0.010445


I0000 00:00:1746021247.321622    9278 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


BFGS:    2 15:54:07      -13.423396        0.000000
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -15.385767        0.015256
BFGS:    1 15:54:07      -15.385772        0.009928
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -16.244947        0.014313
BFGS:    1 15:54:07      -16.244952        0.009818
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -16.474567        0.012627
BFGS:    1 15:54:07      -16.474571        0.009269
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -16.331363        0.008896
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -15.981113        0.005521
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -15.478217        0.003084
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -14.886229        0.001423
      Step     Time          Energy          fmax
BFGS:    0 15:54:07      -14

{'evaluation__structures': [Atoms(symbols='Fe2', pbc=True, cell=[2.2640000000000002, 2.2640000000000002, 2.2640000000000002], initial_magmoms=..., calculator=TPCalculator(...)),
  Atoms(symbols='Fe2', pbc=True, cell=[2.3771999999999998, 2.3771999999999998, 2.3771999999999998], initial_magmoms=..., calculator=TPCalculator(...)),
  Atoms(symbols='Fe2', pbc=True, cell=[2.4904, 2.4904, 2.4904], initial_magmoms=..., calculator=TPCalculator(...)),
  Atoms(symbols='Fe2', pbc=True, cell=[2.6035999999999997, 2.6035999999999997, 2.6035999999999997], initial_magmoms=..., calculator=TPCalculator(...)),
  Atoms(symbols='Fe2', pbc=True, cell=[2.7168, 2.7168, 2.7168], initial_magmoms=..., calculator=TPCalculator(...)),
  Atoms(symbols='Fe2', pbc=True, cell=[2.83, 2.83, 2.83], initial_magmoms=..., calculator=TPCalculator(...)),
  Atoms(symbols='Fe2', pbc=True, cell=[2.9432, 2.9432, 2.9432], initial_magmoms=..., calculator=TPCalculator(...)),
  Atoms(symbols='Fe2', pbc=True, cell=[3.0564000000000004, 3

In [7]:
fmax=0.01
max_steps=10000
properties=("energy", "forces", "stresses", "volume")

@Workflow.wrap.as_macro_node("v0", "e0", "B", "volumes", "energies")
def eos_volume_scan(
    wf,
    base_structure,
    calc,
    calc_kwargs = {"fmax": 0.01, "max_steps": 10000, "properties": ("energy", "forces", "stresses", "volume")},
    axes=["x", "y", "z"],
    strain_range=(-0.2, 0.2),
    num_points=11,
):
    """
    Macro node to perform an EOS volume scan:
      - Generate strained structures
      - Evaluate energies, forces, stresses, and volumes
      - Fit a Birch–Murnaghan equation of state

    Returns:
        v0: Equilibrium volume
        e0: Minimum energy
        B: Bulk modulus
        volumes: List of volumes from the scan
        energies: List of energies from the scan
    """
    # Generate strained structures
    wf.structures_list = generate_structures(
        base_structure,
        axes=axes,
        strain_range=strain_range,
        num_points=num_points,
    )

    # Evaluate structures
    wf.evaluation = evaluate_structures(
        structures=wf.structures_list,
        calc=calc,
        calc_kwargs=calc_kwargs,
    )

    # Extract energies and volumes
    wf.energies = extract_values(
        wf.evaluation.outputs.results_dict,
        key="energy",
    )
    wf.volumes = extract_values(
        wf.evaluation.outputs.results_dict,
        key="volume",
    )

    # Fit equation of state
    wf.eos = equation_of_state(
        wf.energies,
        wf.volumes,
        eos="birchmurnaghan",
    )

    # Return EOS parameters and raw data
    return (
        wf.eos.outputs.v0,
        wf.eos.outputs.e0,
        wf.eos.outputs.B,
        wf.volumes,
        wf.energies,
    )

In [8]:
@pwf.as_function_node("a0")
def get_equil_lat_param(eos_output):
    a0 = eos_output**(1/3)
    return a0

from ase.build import bulk

@pwf.as_function_node("equil_struct")
def get_bulk_structure(name: str,
                        crystalstructure = None,
                        a = None,
                        b = None,
                        c = None,
                        alpha = None,
                        covera = None,
                        u = None,
                        orthorhombic = False,
                        cubic = False,
                        basis=None):
    equil_struct = bulk(name = name,
                        crystalstructure = crystalstructure,
                        a = a,
                        b = b,
                        c = c,
                        alpha = alpha,
                        covera = covera,
                        u = u,
                        orthorhombic = orthorhombic,
                        cubic = cubic,
                        basis = basis)
    return equil_struct

In [9]:
# 1. Prepare calculator and workflow
calc = TPCalculator("/cmmc/u/hmai/FePotentials/potential_files/GRACE/2025_04_28_Fe_FeC_C_pyxtal/final_model/")
wf   = Workflow("EOS")

# 2. Build your base Fe structure
bulk_Fe = bulk("Fe", cubic=True, a=2.83)
bulk_Fe.rattle()

# 3. Attach the macro node to the workflow, capturing all outputs
wf.eos = eos_volume_scan(
    base_structure = bulk_Fe,
    calc           = calc,
    calc_kwargs = {"fmax": 0.01, "max_steps": 10000, "properties": ("energy", "forces", "stresses", "volume")},
    axes           = ["x", "y", "z"],
    strain_range   = (-0.02, 0.02),
    num_points     = 11,
)
wf.a0 = get_equil_lat_param(wf.eos.outputs.v0)
wf.eq_bulk_struct = get_bulk_structure(name = "Fe",
                           cubic = True,
                           a = wf.a0)
wf.run()

      Step     Time          Energy          fmax
BFGS:    0 15:54:12      -16.426848        0.013886
BFGS:    1 15:54:12      -16.426853        0.009742
      Step     Time          Energy          fmax
BFGS:    0 15:54:12      -16.445536        0.013684
BFGS:    1 15:54:12      -16.445541        0.009676
      Step     Time          Energy          fmax
BFGS:    0 15:54:12      -16.459238        0.013455
BFGS:    1 15:54:12      -16.459243        0.009596
      Step     Time          Energy          fmax
BFGS:    0 15:54:12      -16.468379        0.013201
BFGS:    1 15:54:12      -16.468384        0.009501
      Step     Time          Energy          fmax
BFGS:    0 15:54:12      -16.473365        0.012925
BFGS:    1 15:54:12      -16.473369        0.009392
      Step     Time          Energy          fmax
BFGS:    0 15:54:12      -16.474567        0.012627
BFGS:    1 15:54:12      -16.474571        0.009269
      Step     Time          Energy          fmax
BFGS:    0 15:54:12      -

{'eos__e0': -16.474689854766503,
 'eos__B': 1.0888794134964814,
 'eos__volumes': [21.332292682904004,
  21.594572051010047,
  21.858992463340865,
  22.12556262332826,
  22.39429123440404,
  22.665187000000007,
  22.938258623547974,
  23.21351480847975,
  23.490964258227127,
  23.770615676221958,
  24.052477765896],
 'eos__energies': [-16.426852917139826,
  -16.445540896964737,
  -16.45924258740174,
  -16.468383524321588,
  -16.473368947210723,
  -16.474570704126037,
  -16.472319307811315,
  -16.46690065609176,
  -16.458556567783948,
  -16.447488114746896,
  -16.43386071411896],
 'eq_bulk_struct__equil_struct': Atoms(symbols='Fe2', pbc=True, cell=[2.8281575503078877, 2.8281575503078877, 2.8281575503078877], initial_magmoms=...)}

In [12]:
print(wf.eq_bulk_struct.outputs.equil_struct.value)

Atoms(symbols='Fe2', pbc=True, cell=[2.8281575503078877, 2.8281575503078877, 2.8281575503078877], initial_magmoms=...)


In [ ]:
a = eos.outputs.v0.value**(1/3)

In [ ]:
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
# Plot relative energy vs. volume
plt.plot(wf.volumes.outputs.output.value, wf.energies.outputs.output.value, marker="x")

plt.grid()
plt.legend()
plt.xlabel('Volume (A^3)')
plt.ylabel('Energy (eV)')
plt.title('Volume vs. Relative Energy for Different Fe Potentials')


In [ ]:
def calc_structure(structure, calc, fmax=0.01, max_steps = 10000):
    from ase.optimize import BFGS  # Import the optimizer
    # Convert the pymatgen structure back to ASE atoms for calculation
    atoms = AseAtomsAdaptor.get_atoms(structure)
    atoms.calc = calc

    # Perform ionic relaxation using BFGS optimizer
    optimizer = BFGS(atoms)  # You can also try LBFGS or other optimizers
    converged = optimizer.run(fmax=fmax, steps=max_steps)  # Stop when forces are below 0.05 eV/Å

    # Calculate the potential energy after relaxation
    energy = atoms.get_potential_energy()
    forces = atoms.get_forces()
    stresses = atoms.get_stresses()
    return atoms, energy, converged


def calc_structure_static(structure, calc):
    atoms = AseAtomsAdaptor.get_atoms(structure)
    atoms.calc = calc

    # Calculate the potential energy after relaxation
    energy = atoms.get_potential_energy()
    forces = atoms.get_forces()
    return energy, forces